# <font color=blue><div align="center">Seats Allocation within an Aircraft</div></font>



## Modules

In [1]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

## Préprocessing des données

Pour la suite de notre étude, on considère le modèle d'avion **Airbus A320**, qui dispose de **174** sièges. (La classe businees n'est pas prise en compte)


<img src="modele_avion.jpg" alt="Modèle de l'avion" width="400" height="600">

In [2]:
number_of_seats = 174

In [3]:
# Load the Excel file into a pandas DataFrame

df_21Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=0 , skipfooter=2)
df_22Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=1 , skipfooter=2)
df_23Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=2 , skipfooter=2)
df_24Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=3 , skipfooter=2)
df_30Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=4 , skipfooter=2)
df_05Nov = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=5 , skipfooter=2)
df_07Nov = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=6 , skipfooter=2)


In [4]:
# On choisit la df sur laquelle on va travailler

df = df_21Oct

In [5]:
df.head()

,Numéro du groupe,Femmes,Hommes,WCHR,TransitTime
0,1,NaN,1.0,NaN,01:15:00
1,2,1.0,NaN,NaN,00:00:00
2,3,NaN,1.0,NaN,00:00:00
3,4,1.0,NaN,NaN,00:00:00
4,5,NaN,2.0,NaN,05:25:00


In [6]:
def convert_TransitTime(time_val):

    # Convert time to minutes
    total_minutes = time_val.hour * 60 + time_val.minute
    
    # Check if total minutes is greater than 2 hours or equal to 0
    if total_minutes > 120 or total_minutes == 0:
        return float('inf')  # Return infinity
    else:
        return total_minutes

df['TransitTime'] = df['TransitTime'].apply(convert_TransitTime)


In [7]:
df

,Numéro du groupe,Femmes,Hommes,WCHR,TransitTime
0,1,NaN,1.0,NaN,75.0
1,2,1.0,NaN,NaN,inf
2,3,NaN,1.0,NaN,inf
3,4,1.0,NaN,NaN,inf
4,5,NaN,2.0,NaN,inf
...,...,...,...,...,...
106,107,1.0,NaN,NaN,inf
107,108,2.0,NaN,NaN,inf
108,109,2.0,NaN,NaN,inf
109,110,2.0,NaN,NaN,95.0


In [8]:
# Groups : dict[int : List[int]]

Passagers = dict()
i = 1

for group in df.itertuples():
    if not pd.isna(group[2]):
        for k in range(int(group[2])):
            Passagers[i] = {'gender': 0, 'group':group[1], 'transit':group[5]}
            i+=1
    if not pd.isna(group[3]):
        for k in range(int(group[3])):
            Passagers[i] = {'gender': 1, 'group':group[1], 'transit':group[5]}
            i+=1
    if not pd.isna(group[4]):
        for k in range(int(group[4])):
            Passagers[i] = {'gender': 2, 'group':group[1], 'transit':group[5]}
            i+=1
    


    
Passagers  

{1: {'gender': 1, 'group': 1, 'transit': 75.0},
 2: {'gender': 0, 'group': 2, 'transit': inf},
 3: {'gender': 1, 'group': 3, 'transit': inf},
 4: {'gender': 0, 'group': 4, 'transit': inf},
 5: {'gender': 1, 'group': 5, 'transit': inf},
 6: {'gender': 1, 'group': 5, 'transit': inf},
 7: {'gender': 0, 'group': 6, 'transit': inf},
 8: {'gender': 0, 'group': 7, 'transit': inf},
 9: {'gender': 0, 'group': 8, 'transit': 80.0},
 10: {'gender': 1, 'group': 8, 'transit': 80.0},
 11: {'gender': 1, 'group': 9, 'transit': inf},
 12: {'gender': 0, 'group': 10, 'transit': inf},
 13: {'gender': 0, 'group': 11, 'transit': inf},
 14: {'gender': 1, 'group': 11, 'transit': inf},
 15: {'gender': 1, 'group': 12, 'transit': 70.0},
 16: {'gender': 1, 'group': 13, 'transit': inf},
 17: {'gender': 0, 'group': 14, 'transit': inf},
 18: {'gender': 1, 'group': 14, 'transit': inf},
 19: {'gender': 1, 'group': 15, 'transit': inf},
 20: {'gender': 1, 'group': 16, 'transit': 95.0},
 21: {'gender': 0, 'group': 17, 'tr

## Model 

In [9]:
number_of_seats = 174
weight_f = 70
weight_m = 85
weight_h = 92.5
number_of_rows = 29
number_of_columns = 6

number_f = int(sum([x for x in df['Femmes'] if not pd.isna(x)]))
number_m = int(sum([x for x in df['Hommes'] if not pd.isna(x)]))
number_h = int(sum([x for x in df['WCHR'] if not pd.isna(x)]))

In [10]:
m = Model("Seats_Allocation")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-04


**Définition des variables**

In [11]:
AssignmenVarDict = {(i, j, k): m.addVar(vtype=GRB.BINARY, name=f'x_{i}_{j}_{k}') 
                    for i in range(1, number_of_rows + 1) 
                    for j in range(1, number_of_columns + 1) 
                    for k in range(1, len(Passagers) + 1)}

**Contrainte Satisfaction Clients**

In [12]:
# Contrainte : tous les passagers ont un siège 

m.addConstr(quicksum([AssignmenVarDict[(i, j, k)] 
                      for i in range(1, number_of_rows + 1) 
                      for j in range(1, number_of_columns + 1) 
                      for k in range(1, len(Passagers) + 1)]) 
            == len(Passagers), name='SeatPassengersCONST')

<gurobi.Constr *Awaiting Model Update*>

In [13]:
#Contrainte : Siège par personne

SeatPerPassengerCONST = {k: m.addConstr(quicksum([AssignmenVarDict[(i, j, k)] 
                                                  for i in range(1, number_of_rows + 1) 
                                                  for j in range(1, number_of_columns)]) 
                                         == 1, name=f'Passenger{k}UniqueSeat') 
                         for k in range(1, len(Passagers) + 1)}

In [14]:
#Contrainte : Personne par siège

SeatPerPassengerCONST = {(i,j): m.addConstr(quicksum([AssignmenVarDict[(i, j, k)] 
                                                      for k in range(1, len(Passagers) + 1) ]) 
                                         <= 1, name=f'Seat({i},{j})UniquePassenger') 
                         for i in range(1, number_of_rows + 1)
                         for j in range(1, number_of_columns +1)}



**Contrainte Centrage de L'avion**

In [15]:
PondeMen_i = weight_m * quicksum([AssignmenVarDict[(i, j, k)] * i 
                                 for i in range(1, number_of_rows + 1) 
                                 for j in range(1, number_of_columns + 1) 
                                 for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 1]) 

PondeWomen_i = weight_f * quicksum([AssignmenVarDict[(i, j, k)] * i 
                                   for i in range(1, number_of_rows + 1) 
                                   for j in range(1, number_of_columns + 1) 
                                   for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 0]) 

PondeWCHR_i = weight_h * quicksum([AssignmenVarDict[(i, j, k)] * i 
                                  for i in range(1, number_of_rows + 1) 
                                  for j in range(1, number_of_columns + 1) 
                                  for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 2]) 

Barycentre_i = (PondeMen_i + PondeWCHR_i + PondeWomen_i) / (weight_f * number_f + weight_h * number_h + weight_m * number_m)

In [16]:
PondeMen_j = weight_m * quicksum([AssignmenVarDict[(i, j, k)] * j 
                                 for i in range(1, number_of_rows + 1) 
                                 for j in range(1, number_of_columns + 1) 
                                 for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 1]) 

PondeWomen_j = weight_f * quicksum([AssignmenVarDict[(i, j, k)] * j 
                                   for i in range(1, number_of_rows + 1) 
                                   for j in range(1, number_of_columns + 1) 
                                   for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 0]) 

PondeWCHR_j = weight_h * quicksum([AssignmenVarDict[(i, j, k)] * j 
                                  for i in range(1, number_of_rows + 1) 
                                  for j in range(1, number_of_columns + 1) 
                                  for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 2]) 

Barycentre_j = (PondeMen_j + PondeWCHR_j + PondeWomen_j) / (weight_f * number_f + weight_h * number_h + weight_m * number_m)

In [17]:
m.addConstr(Barycentre_i <= 17, name='Bary_i_Sup')
m.addConstr(Barycentre_i >= 13, name='Bary_i_Inf')
m.addConstr(Barycentre_j <= 4, name='Bary_j_Sup')
m.addConstr(Barycentre_j >= 3, name='Bary_j_Inf')

<gurobi.Constr *Awaiting Model Update*>

**Contrainte WCHR**

In [18]:
'''SeatPer_WCHR_CONST = {k: m.addConstr(quicksum([AssignmenVarDict[i, j, k] 
                                               for i in range(1, number_of_rows) 
                                               for j in [3, 5] 
                                               ]) == 1, name=f"SeatPerWCHR{k}")
                                               for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 2}'''

'SeatPer_WCHR_CONST = {k: m.addConstr(quicksum([AssignmenVarDict[i, j, k] \n                                               for i in range(1, number_of_rows) \n                                               for j in [3, 5] \n                                               ]) == 1, name=f"SeatPerWCHR{k}")\n                                               for k in range(1, len(Passagers) + 1) if Passagers[k][\'gender\'] == 2}'

In [19]:
'''for k in range(1, len(Passagers) + 1):
    if Passagers[k]['gender'] == 2:
        for i in range(1, number_of_rows):
            for j in [3, 5]:
                m.addConstr((AssignmenVarDict[i, j, k] + AssignmenVarDict[i, j - 1, k]) <= 1, 
                            name=f'Neighbor_WCHR{k}_{i}_{j-1}')
                m.addConstr((AssignmenVarDict[i, j, k] + AssignmenVarDict[i + 1, j, k]) <= 1, 
                            name=f'Neighbor_WCHR{k}_{i+1}_{j}')
                m.addConstr((AssignmenVarDict[i, j, k] + AssignmenVarDict[i + 1, j - 1, k]) <= 1, 
                            name=f'Neighbor_WCHR{k}_{i+1}_{j-1}')'''

"for k in range(1, len(Passagers) + 1):\n    if Passagers[k]['gender'] == 2:\n        for i in range(1, number_of_rows):\n            for j in [3, 5]:\n                m.addConstr((AssignmenVarDict[i, j, k] + AssignmenVarDict[i, j - 1, k]) <= 1, \n                            name=f'Neighbor_WCHR{k}_{i}_{j-1}')\n                m.addConstr((AssignmenVarDict[i, j, k] + AssignmenVarDict[i + 1, j, k]) <= 1, \n                            name=f'Neighbor_WCHR{k}_{i+1}_{j}')\n                m.addConstr((AssignmenVarDict[i, j, k] + AssignmenVarDict[i + 1, j - 1, k]) <= 1, \n                            name=f'Neighbor_WCHR{k}_{i+1}_{j-1}')"

## Fonction objectif

**Fonction obj : groupes**

In [20]:
a = 0.3  # Poids pour la distance en ligne
b = 0.1  # Poids pour la distance en colonne

objective_groups = quicksum((a * abs(i1 - i2) + b * abs(j1 - j2)) * AssignmenVarDict[(i1, j1, k1)] * AssignmenVarDict[(i2, j2, k2)]
                            for i1 in range(1, number_of_rows + 1)
                            for j1 in range(1, number_of_columns + 1)
                            for i2 in range(1, number_of_rows + 1)
                            for j2 in range(1, number_of_columns + 1)
                            for k1 in Passagers.keys()
                            for k2 in Passagers.keys()
                            if Passagers[k1]['group'] == Passagers[k2]['group'])

**Fonction obj : transit**

In [21]:
objective_transit = quicksum(AssignmenVarDict[(i, j, k)] *(i / (Passagers[k]['transit']))
                             for i in range(1, number_of_rows + 1) 
                             for j in range(1, number_of_columns + 1) 
                             for k in range(1, len(Passagers) + 1) 
                             if Passagers[k]['transit'] != float('inf'))

In [22]:
#obj = objective_transit + objective_groups

## Optimisation

In [23]:
# Ajout de la fonction objectif
m.setObjective(objective_groups, GRB.MINIMIZE)

m.params.outputflag = 0 # mode muet

# Mise à jour du modèle  --
m.update()

In [24]:
# -- Résolution --
m.optimize()

In [25]:

# -- Vérification du statut et Affichage (le cas échéant) des solutions --
if m.status == GRB.INF_OR_UNBD:
    m.setParam(GRB.Param.Presolve, 0)
    m.optimize()

# Vérification du statut et affichage des solutions si nécessaire
if m.status == GRB.INF_OR_UNBD:
    m.setParam(GRB.Param.Presolve, 0)
    m.optimize()

if m.status == GRB.INFEASIBLE:
    print("Feasible solution found, but not necessarily optimal.")
elif m.status == GRB.UNBOUNDED:
    print("Model is unbounded.")
else:
    print(f'Optimal solution found with satisfaction = {round(m.objVal, 2)}')

print()

Feasible solution found, but not necessarily optimal.

